# 고성능 대규모 언어 모델 기반 리랭킹
- 고성능 대규모 언어 모델 기반 리랭킹은 Claude나 GPT와 같은 고성능 언어 모델을 활용하여 초기 검색 결과를 재평가하고 순위를 조정하는 방식입니다.
- 구현 방식
1. 프롬프트 설계: 질의와 문서를 입력으로 받아 관련성을 평가하는 프롬프트를 작성합니다.
2. 대규모 언어 모델 호출: 각 문서에 대해 대규모 언어 모델을 호출하여 관련성 점수를 얻습니다.
3. 순위 재조정: 얻은 점수를 기반으로 문서들의 순위를 재조정합니다.
- 이 방식은 고성능 대규모 언어 모델을 활용하기 때문에 정확한 관련성 평가가 가능합니다.
- 광범위하게 학습된 대규모 언어모델 특성상, 추가적인 학습 없이 다양한 도메인의 질의 유형에 쉽게 적용할 수 있다는 점도 이 방식의 장점으로 꼽을 수 있습니다.

- 필요시 관련성 평가 점수 추론 근거를 제공할 수 있기 때문에 결과해석이 용이하다는 특징이 있습니다.
- 반면, 많은 리소스를 필요로 하기 때문에 계산 비용이 높다는 점과 느린 처리 속도는 단점으로 꼽힙니다.

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "data/투자설명서.pdf"
loader = PyPDFLoader(file_path)
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)
docs = loader.load_and_split(doc_splitter)

In [2]:
from langchain_ollama import OllamaEmbeddings
embedding = OllamaEmbeddings(model="bge-m3")

- FAISS 데이터베이스를 생성하고 저장합니다.

In [3]:
from langchain_community.vectorstores import FAISS

faiss_store = FAISS.from_documents(docs, embedding)
persist_directory = "data/DB"
faiss_store.save_local(persist_directory)

In [4]:
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

- 이제 대규모 언어 모델 기반 리랭킹 알고리즘을 생성합니다.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from textwrap import dedent
from langchain_core.output_parsers import JsonOutputParser

- BaseModel, Field: Pydantic 라이브러리에서 제공하는 클래스로, 데이터 모델을 정의하고 검증하는데 사용합니다.
- PromptTemplate: 랭체인에서 제공하는 클래스로, 프롬프트 템플릿을 생성하는데 사용합니다.
- Document: 랭체인의 문서 클래스로, 텍스트 내용과 메타데이터를 포함하는 문서 객체를 표현합니다.
- List, Dict, Any, Tuple: 파이썬 typing 모듈에서 제공하는 타입 힌트로, 함수의 입력과 출력 타입을 명시하는데 사용합니다.
- ChatOpenAI: 랭체인에서 챗GPT 모델을 사용하기 위한 클래스입니다.
- dedent: textwrap 모듈의 함수로, 문자열의 들여쓰기를 제거하는 데 사용합니다.
- JsonOutputParser: 랭체인에서 제공하는 클래스로, 대규모 언어 모델의 출력을 JSON 형식으로 파싱하는데 사용합니다.

In [30]:
class RelavanceScore(BaseModel):
    relevance_score: float = Field(description="문서가 쿼리와 얼마나 관련이 있는지를 나타내는 점수.")

def reranking_documents(query: str, docs: List[Document], top_n: int = 2) -> List[Document]:
    parser = JsonOutputParser(pydantic_object=RelavanceScore)
    human_message_prompt = PromptTemplate(
        template="""
        1점부터 10점까지 점수를 매겨, 다음 문서가 질문이 얼마나 관련이 있는지 평가해주세요.
        단순히 키워드가 일치하는 것이 아니라 쿼리의 구체적인 맥락과 의도를 고려하세요.
        {format_instructions}
        question: {query}
        document: {doc}
        relevance_score:""",
        input_variables=["query", "doc"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=3000)
    chain = human_message_prompt | llm | parser
    scored_docs = []

    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        try:
            score = chain.invoke(input_data)['relevance_score']
            score = float(score)            
        except Exception as e:
            print(f"오류 발생: {str(e)}")
            default_score = 5 # 기본 점수를 5점으로 설정
            print(f"기본 점수 {default_score}점을 사용합니다.")
            score = default_score        
        scored_docs.append((doc, score))

    reranked_dcos = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_dcos[:top_n]]

- RelevanceScore클래스는 Pydantic의 BaseModel을 상속받아 관련성 점수를 나타내는 데이터 모델을 정의합니다.
- Pydantic의 BaseModel은 데이터 검증 및 직렬화를 자동으로 처리하여, 입력 데이터가 올바른 형식과 타입으로 유지되도록 보장합니다.
- reranking_documents함수는 사용자의 질문과 초기 검색 문서들을 입력받고, 리랭킹이 수행된 문서 리스트를 반환하는 역할을 수행합니다.
- JsonOutputParser를 사용하여 대규모 언어 모델의 출력을 RelevanceScore 객체로 파싱할 수 있도록 준비합니다.

In [31]:
query = "이 회사의 2022년 영업손실이 정확히 얼마야?"
initial_docs = vectordb.similarity_search(query, k=4)
reranked_docs = reranking_documents(query, initial_docs)

- 사용자의 질문을 정의한 후 FAISS벡터 DB기반으로 밀집검색을 수행합니다.
- reranking_documents 함수에서 리랭킹된 문서 리스트를 반환하도록 합니다.

In [33]:
# 4개의 초기 검색 결과 출력
print(f"Query: {query}\n\n")

print("Top initial documents:")
for i, doc in enumerate(initial_docs):
    print(f"Document {i+1}:")
    print(doc.page_content)

# 리랭킹 결과 출력
print("\n\nTop reranked documents:")
for i, doc in enumerate(reranked_docs):
    print(f"Document {i+1}:")
    print(doc.page_content)

Query: 이 회사의 2022년 영업손실이 정확히 얼마야?


Top initial documents:
Document 1:
하여 2021년 영업손실 130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원, 2024년
1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면에서도, 금융비용 등의 발생 영향
으로 인해 2021년 당기순손실 130.7억원, 2022년 당기순손실 228.7억원, 2023년 당기순손실
116.1억원, 2024년 1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면하지 못하고
있습니다.따라서 당사의 파이프라인에서 임상 성공을 통한 기술이전, 상품화 성공 등의 성과를 이루
Document 2:
업의 성패를 장담할 수 없고, 신규사업 진출이 당사에 악영향을
미치는 결과를 초래할 수 있습니다. 투자자 여러분들께서는 이 점을
반드시 유의하여 주시기 바랍니다.
회사위험
[가. 매출부진 및 지속적 손실 발생 위험]
당사는 매출이 부진한 가운데, 지속적으로 신약후보물질의 발굴과 보유 파
이프라인의 적응증 증가를 위하여 다양한 비임상 및 임상시험을 준비하고
있으며 이에 따라 관련비용의 지출이 꾸준히 발생하여 2021년 영업손실
130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원,
Document 3:
외적 측면에서도, 금융비용 등의 발생 영향으로 인해 2021년 당기순손실 130.7억원, 2022년
당기순손실 228.7억원, 2023년 당기순손실 116.1억원, 2024년 1분기 당기순손실 32.9억원
이 발생하는 등 지속적인 적자 구조를 면하지 못하고 있습니다. 따라서 당사의 파이프라인에
서 임상 성공을 통한 기술이전, 상품화 성공 등의 성과를 이루어내지 못한다면 당사의 적자
구조를 개선하는 것은 불가능할 수 있으며, 자본력이 지속적으로 감소하여 지속적인 연구개
Document 4:
있으며 이에 따라 관련비용의 지출이 꾸준히 발생하여 2021년 영업손실
130.1

In [34]:
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

# CustomRetriever 체인을 생성
class CustomRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description="Retrival을 위한 벡터 저장소")
    class Config:
        arbitrary_types_allowed = True

    # num_docs 파라미터 리랭킹 후 반환할 최종 문서의 수를 정의
    def _get_relevant_documents(self, query: str, num_docs=2) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=4)
        return reranking_documents(query, initial_docs, top_n=num_docs)       

- CustomRetriever는 벡터 저장소에서 초기 검색 결과를 가져온 후, 리랭킹을 통해 최종적으로 관련성이 높은 문서만을 선별합니다.
- vectorstore는 검색에 사용되는 벡터 데이터베이스로, 여기서는 FAISS를 사용할 수 있도록 설계되었습니다.
- get_relevant_documents() 메서드는 쿼리에 대한 초기 검색 결과를 가져온 후, reranking_documents 함소를 사용해 관련 문서를 재정렬합니다.

In [36]:
# CustomRetriever 인스턴스 생성
custom_retriever = CustomRetriever(vectorstore=vectordb)

# 답변용 LLM 인스턴스 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True,
)

In [37]:
qa_chain.invoke("이 회사의 2022년 영업손실이 정확히 얼마야?")

{'query': '이 회사의 2022년 영업손실이 정확히 얼마야?',
 'result': '2022년 영업손실은 149.1억원입니다.',
 'source_documents': [Document(id='67623cba-e3f3-43f6-ba02-c2cbec0d17c3', metadata={'producer': 'iText® 5.5.9 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2024-06-26T16:15:14+09:00', 'moddate': '2024-06-26T16:15:14+09:00', 'source': 'data/투자설명서.pdf', 'total_pages': 514, 'page': 158, 'page_label': '159'}, page_content='하여 2021년 영업손실 130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원, 2024년\n1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면에서도, 금융비용 등의 발생 영향\n으로 인해 2021년 당기순손실 130.7억원, 2022년 당기순손실 228.7억원, 2023년 당기순손실\n116.1억원, 2024년 1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면하지 못하고\n있습니다.따라서 당사의 파이프라인에서 임상 성공을 통한 기술이전, 상품화 성공 등의 성과를 이루'),
  Document(id='5fe7aa43-1914-42cc-8950-ccc3a5332389', metadata={'producer': 'iText® 5.5.9 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2024-06-26T16:15:14+09:00', 'moddate': '2024-06-26T16:15:14+09:00', 'source': 'data/투자설명서.pdf